<a href="https://colab.research.google.com/github/JuanEmmanuel30/-folha_uol_news.ipynb/blob/main/'folha_uol_news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Carregar o dataset
df = pd.read_csv('folha_uol_news.csv')

# 2. Filtrar notícias da seção "Mercado" no 1º trimestre de 2015
df['date'] = pd.to_datetime(df['date'])
mask = (
    (df['section'].str.lower() == 'mercado') &
    (df['date'] >= '2015-01-01') &
    (df['date'] <= '2015-03-31')
)
df_mercado = df[mask].copy()

# 3. Carregar o modelo NER
model_name = "monilouise/ner_pt_br"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

# 4. Extrair entidades ORGANIZAÇÃO das notícias
orgs = []
for text in tqdm(df_mercado['text'], desc="Processando notícias"):
    try:
        entities = ner_pipeline(text)
        orgs += [ent['word'] for ent in entities if ent['entity_group'] == 'ORG']
    except Exception:
        continue

# 5. Criar ranking das organizações mais citadas
orgs_series = pd.Series(orgs)
ranking = orgs_series.value_counts().head(10)

# 6. Visualização
plt.figure(figsize=(10,6))
sns.barplot(x=ranking.values, y=ranking.index, palette="viridis")
plt.title('Top 10 Organizações mais citadas na seção Mercado (1º tri 2015)')
plt.xlabel('Número de citações')
plt.ylabel('Organização')
plt.tight_layout()
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: 'folha_uol_news.csv'